In [8]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount("/content/drive")# Load the Drive helper and mount


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
directory = "/content/drive/My Drive/SLI_2/"

In [10]:
import cv2
import numpy as np
import time

trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]:
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)

  return tracker


def preprocess(img, shave=False, shavedim=(350,500, 500,1000)):
    #if the image is to be cropped make sure the values are sane first then reduce the image down to the new dimentions
    if shave:
        if(shavedim[0] < 0):
            shavedim[0] = 0
        if (shavedim[1] > img.shape[0]):
            shavedim[1] = img.shape[0]
        if (shavedim[2] < 0):
            shavedim[2] = 0
        if (shavedim[3] > img.shape[1]):
            shavedim[3] = img.shape[1]
        img = img[shavedim[0]:shavedim[1],shavedim[2]:shavedim[3]]
    sizexy = [img.shape[1], img.shape[0]]

    #get the appropriate padding on the image to make it square
    padhw = [0,0]
    if(sizexy[0] > sizexy[1]):
        dif = sizexy[0] - sizexy[1]
        border = cv2.copyMakeBorder(img, int(dif/2), int(dif/2), 0, 0, cv2.BORDER_CONSTANT, value=[200, 200, 200])
        padhw[0] = int(((dif/2)/border.shape[0]) * 448)

    elif (sizexy[1] > sizexy[0]):
        dif = sizexy[1] - sizexy[0]
        border = cv2.copyMakeBorder(img, 0, 0, int(dif / 2), int(dif / 2), cv2.BORDER_CONSTANT, value=[200, 200, 200])
        padhw[1] = int(((dif / 2) / border.shape[1]) * 448)
    else:
        border = img

    #resize the image to fit the 448,448 input that yolo requires
    border = cv2.resize(border, (448, 448))

    #yolo requires the image to be fed in by (channel, y,x). Transpose to match that.
    transposed = np.transpose(border, [2, 0, 1])
    return transposed, padhw, shavedim, border



CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

net = cv2.dnn.readNetFromCaffe(directory + "ssd/MobileNetSSD_deploy.prototxt.txt", directory + "ssd/MobileNetSSD_deploy.caffemodel")

vs = cv2.VideoCapture(directory + "videos (1)/cars (1).mp4")
multiTracker = cv2.MultiTracker_create()
writer = None
(w, h) = (None, None)

try:
    prop = cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

multiTrackerList = []

while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    #frame = cv2.resize(frame, (int(frame.shape[1]/2), int(frame.shape[0]/2)))
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if frame is None:
      print("kjh")
      break

    processed, padhw, shavedim, resized = preprocess(frame)
    frame = resized

    # get updated location of objects in subsequent frames
    toDelete = []
    for oneTracker in multiTrackerList:
        (success, box) = oneTracker.update(frame)

        # check to see if the tracking was a success
        if success:
            (x, y, W, H) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + W, y + H),
                (0, 255, 0), 2)
    # if the frame dimensions are empty, grab them
    if w is None or h is None:
        (w, h) = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 0.007843, (448, 448), 127.5)
    net.setInput(blob)
    start = time.time()
    detections = net.forward()
    end = time.time()

    # loop over each of the layer outputs
    for i in np.arange(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with the
    # prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the `confidence` is
        # greater than the minimum confidence
        if confidence > 0.2:
            # extract the index of the class label from the `detections`,
            # then compute the (x, y)-coordinates of the bounding box for
            # the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
            if CLASSES[idx] == "car":
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, (startX, startY, endX - startX, endY - startY))
                multiTrackerList.append(tracker)
                cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx], 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)




    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(directory + "output (1)/cars-ssd.avi", fourcc, 30, (frame.shape[1], frame.shape[0]), True)

        # some information on processing single frame
        if total > 0:
            elap = (end - start)
            print("[INFO] single frame took {:.4f} seconds".format(elap))
            print("[INFO] estimated total time to finish: {:.4f}".format(
                elap * total))
    # write the output frame to disk
    writer.write(frame)

writer.release()
vs.release()


[INFO] 15212 total frames in video
[INFO] single frame took 0.2725 seconds
[INFO] estimated total time to finish: 4145.1110


KeyboardInterrupt: ignored